# SuSiE RSS

Bayesian sum of single-effect (SuSiE) linear regression using z scores

After applying LD_Clumping.ipynb and Region_Extraction.ipynb to select regions that overlap between traits, the current pipeline focuses on SuSiE to do fine mapping of those regions to see if theres something of interest.

To run this notebook follow the example:

```
sos run ~/project/UKBB_GWAS_dev/workflow/SuSiE_test.ipynb \
    --cwd output \
    --region_dir /gpfs/gibbs/pi/dewan/data/UKBiobank/results/region_extraction/f3393_hearing_aid \
    --region_file /gpfs/gibbs/pi/dewan/data/UKBiobank/results/LD_clumping/f3393_hearing_aid/200828_UKBB_Hearing_aid_f3393_hearing_aid_cat.fastGWA.snp_stats.clumped_region \
    --sumstats_path /gpfs/gibbs/pi/dewan/data/UKBiobank/results/FastGWA_results/results_imputed_data/f3393_hearing_aid/*.snp_stats.gz \
    --N 230411 \
    --container_lmm /gpfs/gibbs/pi/dewan/data/UKBiobank/lmm.sif
    $JOB_OPT
```

In [32]:
[global]
# Path to region extraction files
parameter: region_dir = path
#The region file after LD clumping
parameter: region_file = path
parameter: sumstats_path = path
fail_if(not region_file.is_file(), msg = 'Cannot find regions to fine map. Please specify them using ``--region-file`` option.')
# Load all regions of interest. Each item in the list will be a region: (chr, start, end)
regions = [x.strip() for x in open(region_file).readlines()]
regions = [x.replace(' ', '_' ) for x in regions]
#The directory for output files
parameter: cwd = path
## The container with the lmm/marp software. Can be either a dockerhub image or a singularity `sif` file.
#parameter: container_lmm = '/home/hs863/LMM/lmm_v_1_3.sif'
parameter: container_lmm = 'statisticalgenetics/lmm:1.4'
parameter: container_marp = 'gaow/marp'
# Specific number of threads to use
parameter: numThreads = 2

In [37]:
[default_1]
parameter: N = int
input: [(f"{region_dir}/{x}/{sumstats_path:bn}.all_chr_{x}.sumstats.gz", f"{region_dir}/{x}/{sumstats_path:bn}.all_chr_{x}.sample_ld.gz") for x in regions], group_by = 2
output: [f'{cwd}/{x}.{sumstats_path:bnn}.SuSiE_RSS.rds' for x in regions], group_by=1
task: trunk_workers = 1, trunk_size = job_size, walltime = '12h', mem = '20G', cores = numThreads, tags = f'{step_name}_{_output:bn}'
R: container=container_lmm, expand = "${ }", stderr = f'{_output:n}.stderr', stdout = f'{_output:n}.stdout'
    sumstat = read.csv(${_input[0]:r}, sep = '\t', header=T,stringsAsFactors=F)
    ld = read.csv(${_input[1]:r}, sep = '\t', header=T, stringsAsFactors=F)
    ld = as.matrix(ld[,-1])
    print(dim(as.double(sumstat$Z)))
    res = susieR::susie_rss(as.double(sumstat$Z), ld, z_ld_weight = 1/${N}, 
                            L = 10,
                            estimate_residual_variance = TRUE, check_R=F,
                            estimate_prior_variance = TRUE, check_z = F)

    res$pos = as.integer(sumstat$POS)
    res$z = as.double(sumstat$Z)
    res$p = as.double(sumstat$P)
    res$var_names = sumstat$SNP
    
    saveRDS(res, ${_output:r})

In [34]:
[default_2]
output: pip_plot = f"{cwd}/{_input:bn}.png", test = f"{cwd}/{_input:bn}.md", group_by=2
task: trunk_workers = 1, trunk_size = job_size, walltime = '12h', mem = '20G', cores = numThreads, tags = f'{step_name}_{_output:bn}'
R: container=container_lmm, expand = "${ }", stderr = f'{_output[0]:n}.stderr', stdout = f'{_output[0]:n}.stdout'
    res = readRDS(${_input:r})
    png(${_output[0]:r}, width = 8, height=5, unit='in', res=300)
    susieR::susie_plot(res, y= "PIP", pos=list(attr='pos',start=res$pos[1],end=res$pos[length(res$pos)]), add_legend=T)
    dev.off()
R: container=container_lmm, expand = "${ }", stderr = f'{_output[1]:n}.stderr', stdout = f'{_output[1]:n}.stdout'
    res = readRDS(${_input:r})
    cs_mat <- do.call(rbind, lapply(res$sets$cs, 
                           function(x) paste(x,collapse=" ")
                           )
             )

    s <- "${_output[1]:bn}"
    tem <- strsplit(s, "\\.")[[1]][1]
    
    write(dim(cs_mat)[1], ${_output[1]:r})
    write.table(cs_mat, ${_output[1]:r}, append = TRUE, row.names = TRUE)
    
    write("purity", ${_output[1]:r}, append = TRUE)
    write.table(res$sets$purity, ${_output[1]:r}, append = TRUE, row.names = TRUE)
    
    for (i in res$sets$cs) {
        z3 <- cbind(i,res$pip[i])
        colnames(z3) <- c('position', 'PIP')
        z3[order(z3[,2], decreasing = TRUE),]
        
        write("lsmthn", ${_output[1]:r}, append = TRUE)
        write.table(z3, ${_output[1]:r}, append = TRUE, row.names = FALSE, col.names = TRUE)
    }

In [35]:
[default_3]
sep = '\n\n---\n'
cw = '{cwd}/'
input: group_by = 'all'
output: analysis_summary = f'{cwd}/{sumstats_path:bnn}.analysis_summary.md'
python: container=container_lmm, expand = "${ }"

    theme = '''---
    theme: base-theme
    style: |
     p {
       font-size: 24px;
       height: 900px;
       margin-top:1cm;
      }
      img {
        height: 70%;
        display: block;
        margin-left: auto;
        margin-right: auto;
      }
      body {
       margin-top: auto;
       margin-bottom: auto;
       font-family: verdana;
      }
    ---    
    '''
    import numpy
    
    def f7(seq):
        seen = set()
        seen_add = seen.add
        return [x for x in seq if not (x in seen or seen_add(x))]

    text = ""
    sep = '\n\n---\n'
    inp = "${_input:r}".split(" ")
    for i, each in enumerate(inp):
        inp[i] = ".".join(each.split(".")[:-1])

    r = f7("${_input:bn}".split(" "))
    
    num_csets = []
    region_info = []

    for i, each in enumerate(f7(inp)):
        text_temp = ""
        text_temp += "#\n\n Susie RSS {region} \n".format(region=r[i])
        text_temp += "![]({region}.png){sep} \n \n".format(region=r[i], sep=sep)

        f = open(each[1:]+".md", "r")
        num_csets.append(int(f.readline()))
        print(num_csets)
        
        if num_csets[i] > 0:
            f.readline()

            csets = [f.readline().strip("\n").split(" ") for n in range(num_csets[i])]
            for j in range(len(csets)):
                csets[j] = [ x.strip("\"") for x in csets[j]]

            print(csets)

            table = ""
            table += "| credible set | positions | \n"
            table += "| --- | --- | \n"
            for j in range(len(csets)):
                table += "| {} | {} | \n".format(csets[j][0], ", ".join(csets[j][1:]))

            text_temp += table + sep

            f.readline()
            f.readline()
            table = "|  | min.abs.corr | mean.abs.corr | median.abs.corr | \n"
            table += "| --- | --- | --- | --- | \n"
            for j in range(len(csets)):
                row = [ x.strip("\"") for x in f.readline().strip("\n").split(" ")]
                table += "| {} | {} | {} | {} | \n".format(row[0], row[1], row[2], row[3])

            text_temp += table + sep

            for j in range(len(csets)):
                table = ""#"{}".format(csets[j][0]) + "\n"
                table += "| {} position | PIP | \n".format(csets[j][0])
                table += "| --- | --- | \n"
                f.readline() 
                f.readline()

                for pos in csets[j][1:]:
                    row = f.readline().strip("\n").split(" ")
                    table += "| {} | {} | \n".format(row[0], row[1])

                text_temp += table + sep
        region_info.append(text_temp)
            
        
        f.close()
    f = open("${_output}", "w")
    
    cset_order = numpy.argsort(num_csets)
    #cset_order = cser_order.tolist()
    #cset_order.reverse()
    for c in cset_order:
        text += region_info[c]
    
    f.write(theme + text)
    
    f.close()

In [36]:
# Generate analysis report: HTML file, and optionally PPTX file
[default_4]
output: f"{_input['analysis_summary']:n}.html"
sh: container=container_marp, expand = "${ }", stderr = f'{_output:n}.stderr', stdout = f'{_output:n}.stdout'
    node /opt/marp/.cli/marp-cli.js ${_input['analysis_summary']} -o ${_output:a} \
        --title '${region_file:bnn} fine mapping analysis' \
        --allow-local-files
    node /opt/marp/.cli/marp-cli.js ${_input['analysis_summary']} -o ${_output:an}.pptx \
        --title '${region_file:bnn} fine mapping analysis' \
        --allow-local-files